<a href="https://colab.research.google.com/github/AngelaHsu02/Stance-Detection/blob/main/hw3_StanceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

remote_url = "https://raw.githubusercontent.com/hhhuang/nlp2019fall/master/classification/"

r = requests.get(remote_url + "train.csv", allow_redirects=True)
open('stance-detection-train.csv', 'wb').write(r.content)

r = requests.get(remote_url + "test.csv", allow_redirects=True)
open('stance-detection-test.csv', 'wb').write(r.content)


500660

In [ ]:
import csv
from collections import Counter

# Training data

fin = open("/content/stance-detection-train.csv")
# Test data
fin = open("/content/stance-detection-test.csv")


reader = csv.DictReader(fin)
labels = Counter()
targets = Counter()
for line in reader:
    labels[line['Stance']] += 1
    targets[line['Target']] += 1
print(labels)
print(targets)

Counter({'AGAINST': 1014, 'NONE': 490, 'FAVOR': 452})
Counter({'Donald Trump': 707, 'Hillary Clinton': 295, 'Feminist Movement': 285, 'Legalization of Abortion': 280, 'Atheism': 220, 'Climate Change is a Real Concern': 169})


In [ ]:
!pip install transformers==4.28.0
!pip install datasets
!pip install torch
!pip install tqdm
!pip install evaluate
!pip install pandas
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:0

In [ ]:
import csv

from collections import defaultdict, Counter

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoConfig
from transformers import TrainingArguments, Trainer
from transformers import get_scheduler
from transformers import pipeline



import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn

from datasets import Dataset
import evaluate
import pandas as pd
import numpy as np

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") #use gpu or not

# Assignment III 修改區
請修改下方的程式碼，使得 F1 越高越好

In [ ]:
!yes | pip uninstall transformers
!pip install --upgrade transformers
import transformers
from transformers import TextClassificationPipeline

!pip install sentencepiece
import sentencepiece

class StanceDetectionModel:
    def __init__(self):
        self.test_data = "/content/stance-detection-test.csv"
        self.training_data = "/content/stance-detection-train.csv"
        self.model_path = "trained_model"
        self.stance_label_mapping = {'AGAINST': 0, 'NONE': 1, 'FAVOR': 2}
        self.inverted_stance_label_mapping = {v: k for k, v in self.stance_label_mapping.items()}

    def create_model(self, load_trained=False):
        pretrained_model = "cardiffnlp/twitter-roberta-base-2021-124m-topic-multi"
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model, ignore_mismatched_sizes=True) ####***
        self.model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=3, ignore_mismatched_sizes=True)
        # sentiment_task = pipeline("sentiment-analysis", model=pretrained_model, tokenizer=pretrained_model)


    def load_tweet_data(self, filename): #to understand
        """ Data source: https://alt.qcri.org/semeval2016/task6/index.php?id=data-and-tools """
        import csv
        reader = csv.DictReader(open(filename))    # The original released dataset was encoded in ISO-8859-1. The attached version has been changed to utf8 for universality.
        cnt = 0
        for row in reader:
            cnt += 1
            labels = [0.0] * 3
            labels[self.stance_label_mapping[row['Stance']]] = 1.0
            yield {'labels': labels, 'text': row['Tweet']}


    def tokenize_dataset(self, data): #to understand
        return self.tokenizer(data['text'], return_tensors='pt', truncation=True, padding='max_length', max_length=280)

    def compute_metrics(self, eval_pred): #to understand
        logits, labels = eval_pred
        predictions, references = [], []
        for i in range(len(logits)):
            predictions.append(np.argmax(logits[i][:3]))
            references.append(np.argmax(labels[i][:3]))
        #predictions = np.argmax(logits, axis=-1)
        # Following the evaluation of SemEval-2016, only the F-scores of Favour (2) and Against (0) are measured.
        return self.metric.compute(predictions=predictions, references=references, labels=[0, 2], average='macro')

    def train(self):
        #stances = ['Hillary Clinton', 'Feminist Movement', 'Legalization of Abortion', 'Atheism', 'Climate Change is a Real Concern', 'Donald Trump']
        self.create_model()
        load_training_data = lambda: self.load_tweet_data(self.training_data)
        load_test_data = lambda: self.load_tweet_data(self.test_data)
        training_data = Dataset.from_generator(load_training_data).shuffle(seed=42).map(self.tokenize_dataset, batched=True)
        test_data = Dataset.from_generator(load_test_data).map(self.tokenize_dataset, batched=True)

        training_args = TrainingArguments(   ##***改參數
            output_dir="test_trainer",
            evaluation_strategy="epoch",
            num_train_epochs=10,
            per_device_train_batch_size=24,
            per_device_eval_batch_size=24,
        )

        self.metric = evaluate.load("f1")
        self.trainer = Trainer(    #包好的training
            model=self.model,
            args=training_args,
            train_dataset=training_data,
            eval_dataset=test_data,
            compute_metrics=self.compute_metrics,
        )
        self.trainer.train()

    def evaluate(self):
        return self.trainer.evaluate()

    def predict(self, instances):
        # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # self.model = self.model.to(device) #避免RuntimeError: Expected all tensors to be on the same device  #程式碼只將模型移至指定的設備，而不是tokenizer
        # # self.tokenizer = self.tokenizer.to(device) #避免AttributeError: 'RobertaTokenizerFast' object has no attribute 'to'

        # predictions = []
        # with torch.no_grad():#在這個區塊中，關閉梯度計算，以節省內存和計算資源。
        #     for instance in instances: #圈遍歷每個文本
        #         inputs = self.tokenizer.encode_plus(instance["text"], truncation=True, padding=True, return_tensors="pt") #使用tokenizer的encode_plus方法將文本編碼為模型可接受的輸入格式。這一行將文本進行編碼，同時進行截斷和填充，並返回PyTorch tensors。
        #         inputs = {key: tensor.to(device) for key, tensor in inputs.items()} #把輸入的每個鍵值對移至指定的設備。
        #         outputs = self.model(**inputs)#將inputs中的鍵值對作為關鍵字參數傳遞給模型。
        #         logits = outputs.logits #從模型的輸出中獲取預測的標籤（logits）
        #         predicted_label_id = torch.argmax(logits, dim=1) #從logits中選取最大值的索引，即預測的標籤索引。
        #         predicted_label = self.inverted_stance_label_mapping[predicted_label_id.item()] #根據預測的標籤索引，從inverted_stance_label_mapping字典中查找對應的標籤。
        #         predictions.append(predicted_label)
        # return predictions

        pipe = TextClassificationPipeline(model=self.model, tokenizer=self.tokenizer, return_all_scores=True,  device=0 ) #加上device=0，調成用gpu，default 是 -1 ，所以pipeline預設是用CPU
        predictions = []
        for results in pipe([ {"text": instance["text"]} for instance in instances]):
            predictions.append(max(
                [(result['score'], self.inverted_stance_label_mapping[int(result['label'][-1])]) for result in results if result['label'] in ['LABEL_0', 'LABEL_1', 'LABEL_2']])[1])
        return predictions

Found existing installation: transformers 4.28.0
Uninstalling transformers-4.28.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.28.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)?   Successfully uninstalled transformers-4.28.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00


In [ ]:
model = StanceDetectionModel()
model.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2021-124m-topic-multi and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([19, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-d79187a21c02844a/0.0.0. Subsequent calls will reuse this data.


Map:   0%|          | 0/2914 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-5a589aef6d7a3790/0.0.0. Subsequent calls will reuse this data.


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Adam's epsilon for numerical stability.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.590421,0.557150
2,No log,0.653787,0.562771
3,No log,0.683107,0.613187
4,No log,0.893317,0.606712
5,0.298300,1.060656,0.595265
6,0.298300,1.185855,0.600218
7,0.298300,1.397852,0.585162
8,0.298300,1.416598,0.612637
9,0.036900,1.482346,0.606962
10,0.036900,1.502369,0.603855


# Assignment III: 目標1

請修改 `Assignment III 修改區` 那格，使得下方回報之 F1 越高越好

In [ ]:
print(model.evaluate()['eval_f1']) #0.6以上

0.6038553740454242


# Assignment III: 目標2
請修改 `Assignment III 修改區` 那格，使得下方程式碼能順利執行

In [ ]:
model.predict([
    {"text": "I don't trust Hillary"},
    {"text": "I don't like Trump"},
    {"text": "I like Hillary Clinton. We are personal friends. My support for her candidacy is based on the fact that she is brilliantly qualified for the job."}
    ])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


['AGAINST', 'NONE', 'FAVOR']